In [71]:
###Import all modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq
import os
import re
from bs4 import BeautifulSoup as htmltotxt

import sklearn.preprocessing as skl_pre
import sklearn.linear_model as skl_lm
import sklearn.discriminant_analysis as skl_da
import sklearn.neighbors as skl_nb

In [92]:
###Creates dataframe with features

#master_array = [] #append results to array to create data frame
#for filename in os.listdir('season/'):
#  split_name = filename.split('.') #obtain the season and episodes
#  season = split_name[0]   
#  episode = split_name[1]
#  #TODO method to get each line of the script
#  master_array.append([season, episode])
master_dict = dict()
feature_dict = dict()

#top5monica = ['cooking','chandler?','work','joey','boots']
#top5joey = ['dude','casting','chandler?','agent','director']
#top5phoebe = ['la','massage','smelly','yay!','buffay']
#top5rachel = ['mon','barry','joshua','plane','ugh!']
#top5ross = ['julie','dude','mon','justjust','elizabeth']
#top5chandler = ['tulsa','kathy','joe!','gym','ring!']

single_words = ["monica","joey","phoebe","rachel","ross","chandler","dude","crazy","bro","mate","trash"] #bro, mate, trash = trash

Features = ["Number of lines","Number of words","Words per line","Exclamation marks","Question marks","Character",
           "unique_words"]
Features.extend(single_words)
for feat in Features:
    master_dict[feat] = []
    feature_dict[feat] = 0

chars = ['monica','joey','phoebe','rachel','ross','chandler']


for filename in os.listdir('season/'):  
    f = open('season/'+filename, 'r')

    data = htmltotxt(f.read()).get_text() #reads f as html file and converts into txt
    
    name_dict = dict()
    for char in chars:
        name_dict[char] = feature_dict.copy()

    pattern = re.compile(r'\s(?=\w+(?=:))') #store the regex
    result = re.split(pattern, data) # split the script where our pattern matched (pink dot)

    episode_array=[]
    for item in result:
        split_line = item.split(':')
        try:
            character = split_line[0]
            speech = split_line[1]
            episode_array.append([character, speech])
        except:
            pass
    unique_words_char = {'monica': []
              ,'joey': []
              ,'phoebe': []
              ,'rachel': []
              ,'ross': []
              ,'chandler':[]}
    
    i = 0
    while(len(episode_array) > i):
        if episode_array[i][0].lower() not in chars:
            del episode_array[i]
        else:
            character =episode_array[i][0].lower()
            sentence =episode_array[i][1].lower()
            #episode_array[i][1] = re.sub('[()]', '', episode_array[i][1])
            #episode_array[i][1] = re.sub('-[^-]+-', '', episode_array[i][1]) #removes everything within --        
            sentence = sentence.replace("-","").replace("\n"," ").replace("\xa0"," ").replace(",","").replace(".","").replace(":","").replace("'","").replace("opening credits","").replace("closing credits","").replace("ending credits","").replace("commercial break","").replace("\x92","")
            sentence = re.sub("[\(\[].*?[\)\]]", "", sentence) #removes everything withing () and []
            sentence = re.sub('\[.*', '', sentence)
            sentence = re.sub('\(.*', '', sentence)
            sentence = sentence.strip()
            if len(episode_array) == i+1: #last sentence
                sentence = sentence[:len(sentence)-3] #delets "end" in the end
                sentence = sentence.strip()

            words = sentence.split()
            
            
            
            name_dict[character]["Number of lines"] +=1
            name_dict[character]["Number of words"] +=len(words)
            name_dict[character]["Exclamation marks"] += sentence.count('!')
            name_dict[character]["Question marks"] += sentence.count('?')
            
            episode_array[i][1].replace('chan','chandler').replace('chandlers','chandler').replace('mnca','monica').replace('phoe','phoebe').replace('pheebs','phoebe').replace('rache','rachel').replace('rach','rachel')
            for word in words:
                if word not in unique_words_char[character]:
                    unique_words_char[character].append(word)
            
            for single_word in single_words:
                name_dict[character][single_word] += sentence.count(single_word)
            
                
            i +=1
    for char in chars:
        if name_dict[char]["Number of lines"] != 0:
            name_dict[char]["Words per line"] = name_dict[char]["Number of words"]/name_dict[char]["Number of lines"]

        master_dict["Number of lines"].append(name_dict[char]["Number of lines"])
        master_dict["Number of words"].append(name_dict[char]["Number of words"])
        master_dict["Words per line"].append(name_dict[char]["Words per line"])
        master_dict["Exclamation marks"].append(name_dict[char]["Exclamation marks"])
        master_dict["Question marks"].append(name_dict[char]["Question marks"])
        master_dict["Character"].append(char)
        master_dict["unique_words"].append(len(unique_words_char[char]))
        for single_word in single_words:
            master_dict[single_word].append(name_dict[char][single_word])
        
        
        #master_dict["top5monica"].append(name_dict[j]["top5monica"])
        #master_dict["top5joey"].append(name_dict[j]["top5joey"])
        #master_dict["top5phoebe"].append(name_dict[j]["top5phoebe"])
        #master_dict["top5rachel"].append(name_dict[j]["top5rachel"])
        #master_dict["top5ross"].append(name_dict[j]["top5ross"])
        #master_dict["top5chandler"].append(name_dict[j]["top5chandler"])

#print(master_dict)

epinfo = pd.DataFrame.from_dict(master_dict, orient='index')
epinfo.transpose().to_csv('epinfo.csv', index=False)        

    

In [96]:
###Train model
url = 'epinfo.csv'
data = pd.read_csv(url)
#print(data)
x = []
loops = 20
xtraintest = ['Number of lines','Number of words','Words per line','Exclamation marks','Question marks',
                    'monica','joey','phoebe','rachel','ross','chandler','dude','crazy']
model = skl_lm.LogisticRegression(solver = 'lbfgs', max_iter = 6000)
for i in range(loops):

    trainI = np.random.choice(data.shape[0], size = int(len(data)*.9), replace = False)
    trainIndex = data.index.isin(trainI)
    train = data.iloc[trainIndex] #training set
    test = data.iloc[~trainIndex] #test set

    #x='Number of lines','Number of words','Different words used','Words per line','Different words ratio','Exclamation marks','Question marks','Most common 10','Most common 30','Most common 50'
    
    X_train = train[xtraintest]
    Y_train = train['Character']
    X_test = test[xtraintest]
    Y_test = test['Character']

    model.fit(X_train,Y_train)

    x.append(np.mean(model.predict(X_test) == Y_test))
    print(x[i])

xmean = np.mean(x)

print("Total average:",xmean)


0.40875912408759124
0.32116788321167883
0.39416058394160586
0.4744525547445255
0.4306569343065693
0.43795620437956206
0.36496350364963503
0.3722627737226277
0.39416058394160586
0.35766423357664234
0.41605839416058393
0.41605839416058393
0.36496350364963503
0.40875912408759124
0.41605839416058393
0.43795620437956206
0.38686131386861317
0.46715328467153283
0.4306569343065693
0.41605839416058393
Total average: 0.40583941605839424


###### 

In [62]:
#unique words for characters

#def Diff(li1, li2):
#    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
#    return li_dif

commonwords = list(set(words_char['monica']) & set(words_char['joey']) & set(words_char['phoebe']) & set(words_char['rachel']) & set(words_char['ross']) & set(words_char['chandler']))
#monicauniqeDiff(words_char['monica'], commonwords)
monicaunique = list(set(words_char['chandler']).symmetric_difference(set(commonwords)))

commonmonica = dict()
for i in words_char['chandler']:
    if i in monicaunique:
        if i in commonmonica:
            commonmonica[i] +=1
        else:        
            commonmonica[i] = 1
commonmonica = sorted(commonmonica.items(), key=lambda item: item[1])

top5monica = ['cooking','chandler?','work','joey','boots']
top5joey = ['dude','casting','chandler?','agent','director']
top5phoebe = ['la','massage','smelly','yay!','buffay']
top5rachel = ['mon','barry','joshua','plane','ugh!']
top5ross = ['julie','dude','mon','justjust','elizabeth']
top5chandler = ['tulsa','kathy','joe!','gym','ring!']


In [94]:
epinfo[2]

Number of lines           19
Number of words          259
Words per line       13.6316
Exclamation marks          7
Question marks             4
Character             phoebe
unique_words             157
monica                     0
joey                       0
phoebe                     0
rachel                     0
ross                       0
chandler                   0
dude                       0
crazy                      0
bro                        0
mate                       0
trash                      0
Name: 2, dtype: object

TypeError: 'str' object cannot be interpreted as an integer